In [18]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ.get("OPENAI_API_KEY")

In [2]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
import tempfile
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain

# First trial
* Chunk를 나누지 않고 pdf의 page 별로 임베딩하고 저장하는 방식
* 성능이 좋지 않은 듯

## PDF file loader

In [21]:
## 방법1
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("data/전라편.pdf")
pages = loader.load()
len(pages)

851

In [23]:
pages

[Document(page_content='한국지명유래집\n▶▶▶▶전라·제주편\n한\n국\n지\n명\n유\n래\n집\n발간등록번호\n11-1611265-000040-10\n전\n라\n·\n제\n주\n편\n국\n토\n지\n리\n정\n보\n원\nThe Origin  of  Korean  Geographical  Names\n광주광역시\n전라북도\n전라남도\n제주특별자치도\n \n', metadata={'source': 'data/전라편.pdf', 'file_path': 'data/전라편.pdf', 'page': 0, 'total_pages': 851, 'format': 'PDF 1.3', 'title': 'ePapyrus PDF Document', 'author': '', 'subject': '', 'keywords': '', 'creator': 'ePapyrus PDF-Pro 4 Free', 'producer': 'ePapyrus PDF-Pro 4 Free', 'creationDate': '', 'modDate': '', 'trapped': ''}),
 Document(page_content='한국지명유래집\n전라·제주편\n국토지리정보원\n', metadata={'source': 'data/전라편.pdf', 'file_path': 'data/전라편.pdf', 'page': 1, 'total_pages': 851, 'format': 'PDF 1.3', 'title': 'ePapyrus PDF Document', 'author': '', 'subject': '', 'keywords': '', 'creator': 'ePapyrus PDF-Pro 4 Free', 'producer': 'ePapyrus PDF-Pro 4 Free', 'creationDate': '', 'modDate': '', 'trapped': ''}),
 Document(page_content='발간사\n지명은삶의무대인국토의공간을구성하며우리고유의문화와가치를담고있습니다. 급변\n하는사회속에서도문화와역사

In [42]:
# ## 방법2
# from langchain_community.document_loaders import PyPDFLoader

# pdf_filepath = 'data/전라편.pdf'
# loader = PyPDFLoader(pdf_filepath)
# pages = loader.load()

# pages[:10]


[Document(page_content='\u0ad8\nhઁ\u0ab1ಞೠ\n\u0ad8ഐ\n11-1611265-000040-10\n\u0a79\nh\nઁ\u0ab1ಞ\nҴ\nਗThe Origin  of  Korean  Geographical  Names\nҟ\u0ab1ҟ\u09c9द\nب\n \n', metadata={'source': 'data/전라편.pdf', 'page': 0}),
 Document(page_content='\u0ad8\nhઁ\u0ab1ಞ\nਗ\n', metadata={'source': 'data/전라편.pdf', 'page': 1}),
 Document(page_content='\u088e\n߸\n\x01\x0f\u05eeפ\nઓೞӝ ਤ೧ 1958ఠ \u0abaঔ\u0ad1\nਸ ੋधೞৈ 2010۽\n1980\tۈ1983١ې \n\u0ad8\t 1987\x01\x0f\u05eeפ\nۄٮ2008ఠ 2012ө\u0ad1 5֙\n೯\nغ\nח2\r360ৈѐ\n\x0f\u05eeפ\nഝਊ\n\x01\x0f\u05eeפ݀٘\n֢\n\x0f\u05eeפ݀٘\n2010֙12ਘ\nউ\n', metadata={'source': 'data/전라편.pdf', 'page': 2}),
 Document(page_content='ݺ\n\u0cdfҮ\n\x01 ઁ1ݺ\nઁ2ب\n\u0cdfҮ\n ઁ3ೱ\nҟ\u0ab1ҟ\u09c9द\n\nࣗҟ\u0ab1ҟ\u09c9द\nب࠘ۄ\n\u0cdfҮ\nউҵ\n\u0cdfҮ\nद\n\u0cdfҮ\nହҵ\n\u0cdfҮ\n \u0a79\u0ab1द\r\x01৮\u0ab1ҵ\n\u0cdfҮ\n թਗद\n\u0cdfҮ\nҵ\nب\n\n\u088eৈ\u0ab1द\r\x01ҟনद\n\nࣗনҵ\n\u0cdfҮ\nҵ\n\u0cdfҮ\nҵ\n\u0cdfҮ\nҵ\r\x01\u0a62൜ҵ\n\u0cdfҮ\nನद\r\x01\u09d4ঐҵ\r\x01नউҵ\n\u0cdfҮ\n Ҋ൜ҵ\r\x01ъ\u0ad3ҵ\r\x01೧թҵ\n\u0cdfҮ\n

In [12]:
pages = loader.load_and_split()
len(pages)

15

## Embedding vector store

In [13]:
# Embeddings 생성
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=os.environ['OPENAI_API_KEY'])

# Chroma 벡터 스토어 생성
vectorstore = Chroma.from_documents(documents=pages, embedding=embeddings)

In [14]:
# 벡터 스토어에 저장된 벡터의 개수 확인
num_vectors = vectorstore._collection.count()
print(f"Number of vectors in the vectorstore: {num_vectors}")

# 특정 쿼리를 통해 문서 검색
query = "제주도"
retrieved_docs = vectorstore.similarity_search(query)

# 검색된 문서 내용 확인
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}: {doc.page_content}")

Number of vectors in the vectorstore: 15
Document 1: ೠҴ૑ݺਬې૘
੹ۄhઁ઱ಞ
Ҵష૑ܻ੿ࠁਗ
Document 2: ೠҴ૑ݺਬې૘
ÜÜÜÜ ੹ۄhઁ઱ಞ
ೠ
Ҵ
૑
ݺ
ਬ
ې
૘
ߊр ١ ۾ ߣഐ
11-1611265-000040-10
੹
ۄ
h
ઁ
઱
ಞ
Ҵ
ష
૑
ܻ
੿
ࠁ
ਗ
The Origin  of  Korean  Geographical  Names
ҟ ઱ҟ ৉द
੹ۄ ࠘ب
੹ۄ թ ب
ઁ઱ౠ߹੗஖ب
Document 3: ઁAAࠗ
੹ۄ
h
ઁ઱ӂ੄૑ݺ
Document 4: ઁ੢੹ۄhઁ઱ӂ੄૑ܻ৬૑ݺ15

࢑૑৬૑ݺ
	
੹ۄӂ
˝ ࢑ݓ
੹ۄ࠘ب ࠘زױীਤ஖ೠޖ઱ҵ ੌ؀ীࢲթࢲଃਵ۽ ੉য૑ח ֢۸࢑ݓ੉زଃ੄࢑૑
ܳ੉ܘ׮੉࢑ݓ਷୽୒࠘ب ৔زীࢲ੹ۄթب ޖউী੉ܰח ࢑૑۽ ੹୓੸ਵ۽ ੹ۄ ૑
৉ীࣘೠ׮ࣗߔ࢑ݓ	ᖀᎁᒹቲ
੄૑ݓਵ۽ࢲ୶ೂ۸	ᱩᶫᆍ
ࠗӔীࢲթࢲߑೱਵ۽ тۄ
126m઱ਃ࢑਷਍੢࢑	1
݃੉࢑	678m
ղ੢࢑नࢶࠉ	763m
ߑ੢࢑	734m
క୒࢑
	593m
ࠛц࢑	516m
١ਵ۽ ੷࢑ࢿ࢑૑ܳ੉ܘ׮੉࢑ݓҗ ࣗߔ࢑ݓ੄ࢎ੉ীח թࢲߑ
ೱਵ۽ ಩10_20km੄૑ҳ࢚	ᬡཐᓧ
Ҋਗ੉੹ѐغয੓׮੉Ҕਸ҃҅۽ Әъ੉࠘ܨೞ
Ҋ ࢻ૓ъ੉թܨೞৈ૓উҊਗ੉নೞୌ੄࠙ࣻ۸ਸ੉ܘ׮࢑ݓ੄ࢲଃীࢲח ݅҃ъҗ
ز૓ъ੉ߊਗೞݴև਷ഐթಣঠо ੹ѐػ׮࢑ݓ੄թࠗੌ؀ח ز૓ъҗ ৔࢑ъ੄࠙ࣻ҅
ܳ੉ܘ׮	Ӓܿ1 ଵઑ

ઑࢶद؀
~࢑҃಴
ীח ֢۸࢑ݓ੄ੌࠗ઴ӝо ഐթ੿ݓਵ۽ ಴ӝغӝب ೞ৓׮੉࢑ݓ
੉૑Әҗ э੉ة݀ػ ࢑઴ӝ۽ ੋधغӝ द੘ೠѪ਷20ࣁӝ ୡࠗఠ੉׮૑ݺ਷੹ۄ࠘ب
੿਻ҵ ੑঐݶҗ ੹ۄթب ੢ࢿҵ ࠘੉ݶࢎ੉੄ب	ა
҃҅ܳ੉ܖח ֢۸	ᆨᆍ
ীࢲ࠺܃غ
঻׮
޹઱૑࢑ীࢲ૑ܻ࢑ਵ۽ ੉য૑ח ࣗߔ࢑ݓ੄ࢲࢎݶ࢑૑о ੹ۄթب੄زଃ࢑૑ܳഋ
ࢿೠ׮ࣗߔ࢑ݓ਷కߔ࢑ীࢲకߔ࢑ݓҗ тۄઉ޹઱૑࢑ী੉ܰӝ ੹ө૑࢚҃ب৬୽୒
૑ܻ࢑ਸѢ୛ৈࣻ߈بী੉ܲ׮Әъ
৉ࢎ੸ਵ۽ ੹ۄب੄ഐթ ޙചӂҗ ࢚҃ب੄
৔թ ޙചӂਸ

## query, retrieval

In [17]:
# 원하는 모델 이름으로 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(model_name="gpt-4o")  

# 문서 결합 체인을 로드합니다 (QA 체인)
qa_chain = load_qa_chain(llm, chain_type="stuff")

# RetrievalQA 체인 생성
retrieval_qa_chain = RetrievalQA(combine_documents_chain=qa_chain, retriever=vectorstore.as_retriever())

# 예제 질문
query = "제주도의 사질 해안은 전체의 몇 퍼센트에 해당하니?"
response = retrieval_qa_chain.invoke(query)
print(response)


{'query': '제주도의 사질 해안은 전체의 몇 퍼센트에 해당하니?', 'result': '죄송합니다만, 주어진 정보로는 제주도의 사질 해안이 전체의 몇 퍼센트에 해당하는지 알 수 없습니다.'}


In [18]:
# 원하는 모델 이름으로 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(model_name="gpt-4o")  

# 문서 결합 체인을 로드합니다 (QA 체인)
qa_chain = load_qa_chain(llm, chain_type="stuff")

# RetrievalQA 체인 생성
retrieval_qa_chain = RetrievalQA(combine_documents_chain=qa_chain, retriever=vectorstore.as_retriever())

# 예제 질문
query = "제주도의 하천은 어떤 형태를 보이니?"
response = retrieval_qa_chain.invoke(query)
print(response)


{'query': '제주도의 하천은 어떤 형태를 보이니?', 'result': '제주도의 하천은 일반적으로 짧고 경사가 급한 형태를 보입니다. 이러한 특성은 제주도의 화산 지형과 관련이 있습니다. 제주도는 화산 활동으로 형성된 섬으로, 중심부에 한라산이 위치하고 있으며, 이 산에서 시작된 하천들은 짧은 거리 동안 급격히 흘러내려갑니다.'}


# Second trial

## Create Collection

In [52]:
import chromadb
chromadb_client = chromadb.HttpClient(host="localhost", port=8000)
print(chromadb_client.list_collections())

[Collection(name=test_HuggingFace), Collection(name=test_240424), Collection(name=pdf_test_240531)]


In [80]:
# 삭제
chromadb_client.delete_collection(
    name="pdf_test_240531"
)

In [81]:
collection = chromadb_client.create_collection(name="pdf_test_240531")
print(chromadb_client.list_collections())

[Collection(name=test_HuggingFace), Collection(name=test_240424), Collection(name=pdf_test_240531)]


In [82]:
collection = chromadb_client.get_collection("pdf_test_240531")


{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None}

## PDF file loader

In [83]:
## 방법1
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("data/전라편.pdf")
pages = loader.load()
len(pages)

851

In [84]:
documents = pages[:15]
documents

[Document(page_content='한국지명유래집\n▶▶▶▶전라·제주편\n한\n국\n지\n명\n유\n래\n집\n발간등록번호\n11-1611265-000040-10\n전\n라\n·\n제\n주\n편\n국\n토\n지\n리\n정\n보\n원\nThe Origin  of  Korean  Geographical  Names\n광주광역시\n전라북도\n전라남도\n제주특별자치도\n \n', metadata={'source': 'data/전라편.pdf', 'file_path': 'data/전라편.pdf', 'page': 0, 'total_pages': 851, 'format': 'PDF 1.3', 'title': 'ePapyrus PDF Document', 'author': '', 'subject': '', 'keywords': '', 'creator': 'ePapyrus PDF-Pro 4 Free', 'producer': 'ePapyrus PDF-Pro 4 Free', 'creationDate': '', 'modDate': '', 'trapped': ''}),
 Document(page_content='한국지명유래집\n전라·제주편\n국토지리정보원\n', metadata={'source': 'data/전라편.pdf', 'file_path': 'data/전라편.pdf', 'page': 1, 'total_pages': 851, 'format': 'PDF 1.3', 'title': 'ePapyrus PDF Document', 'author': '', 'subject': '', 'keywords': '', 'creator': 'ePapyrus PDF-Pro 4 Free', 'producer': 'ePapyrus PDF-Pro 4 Free', 'creationDate': '', 'modDate': '', 'trapped': ''}),
 Document(page_content='발간사\n지명은삶의무대인국토의공간을구성하며우리고유의문화와가치를담고있습니다. 급변\n하는사회속에서도문화와역사

In [85]:
type(documents)

list

## Chunk Split. Embedding, Vector store

In [86]:
import chromadb
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions

def insert(content):
    client = chromadb.HttpClient(host="localhost", port=8000)
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                model_name="text-embedding-ada-002"
            )
    collection = client.get_collection(name="pdf_test_240531", embedding_function=openai_ef)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    docs = text_splitter.split_documents(content)

    for doc in docs:
        uuid_val = uuid.uuid1()
        print("Inserted documents for ", uuid_val)
        collection.add(ids=[str(uuid_val)], documents=doc.page_content)

In [87]:
insert(documents)

Inserted documents for  b57b82e4-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b5b1c674-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b5d59f40-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b5fe2eec-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b62b5afc-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b656cca0-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b67ffcb0-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b6a92644-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b6d4e234-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b6fbbfbc-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b72d88ee-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b75ba7f6-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b78cb3fa-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b7bc7a36-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b7ea7364-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  b80f2966-1f0c-11ef-9806-ae08445dcffc
Inserted documents for  

In [88]:
collection.peek()

{'ids': ['b57b82e4-1f0c-11ef-9806-ae08445dcffc',
  'b5b1c674-1f0c-11ef-9806-ae08445dcffc',
  'b5d59f40-1f0c-11ef-9806-ae08445dcffc',
  'b5fe2eec-1f0c-11ef-9806-ae08445dcffc',
  'b62b5afc-1f0c-11ef-9806-ae08445dcffc',
  'b656cca0-1f0c-11ef-9806-ae08445dcffc',
  'b67ffcb0-1f0c-11ef-9806-ae08445dcffc',
  'b6a92644-1f0c-11ef-9806-ae08445dcffc',
  'b6d4e234-1f0c-11ef-9806-ae08445dcffc',
  'b6fbbfbc-1f0c-11ef-9806-ae08445dcffc'],
 'embeddings': [[-0.008829634636640549,
   -0.006496971007436514,
   0.021797548979520798,
   -0.021667415276169777,
   -0.017971592023968697,
   0.01808871328830719,
   -0.022903693839907646,
   0.009909751825034618,
   -0.01747708022594452,
   0.003425793955102563,
   -0.013286745175719261,
   0.006851587910205126,
   -0.03383500128984451,
   -0.01640997640788555,
   -0.013768243603408337,
   0.025155022740364075,
   -0.000718587136361748,
   -0.038623955100774765,
   0.009708043187856674,
   -0.0102741289883852,
   0.016357922926545143,
   -0.012922368012368679,


## query, retrieval

In [89]:
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

def queryDB(query):
    client = chromadb.HttpClient(host="localhost", port=8000)
    embedding_function = OpenAIEmbeddings()
    db4 = Chroma(client=client, collection_name="pdf_test_240531", embedding_function=embedding_function)
    docs = db4.similarity_search_with_score(query) ## 유사도 점수까지 확인: https://wikidocs.net/234094
    # retriever = db4.as_retriever(search_type="mmr") ## mmr 검색
    
    return docs # retriever.get_relevant_documents(query)[0]

In [90]:
query="소백산은 어디에 위치하며, 몇개의 도, 시, 군에 걸쳐있니?"
result = queryDB(query)
result

[(Document(page_content='었다. \n민주지산에서지리산으로이어지는소백산맥의서사면산지가전라남도의동쪽산지를형\n성한다. 소백산맥은태백산에서태백산맥과갈라져민주지산에이르기전까지경상도와충청\n도의경계를이루다가, 이후대덕산, 덕유산, 백운산, 지리산을거쳐여수반도에이른다. 금강,\n남한강수계와낙동강수계를나누고있으며, 역사적으로전라도의호남문화권과경상도의\n영남문화권을나누는경계역할을하였다.'),
  0.2784852021654618),
 (Document(page_content='되는데소백산맥은제2기산맥에해당한다. 이후여러차례내용수정이있은뒤\n‘소백산맥’\n은\n지금의지리적범위에해당하는산지를지칭하는지명이되었다. 백두산과지리산을연결하는\n‘백두대간’\n지명이보편적으로사용되면서최근에는산맥지명의타당성에대해논쟁이일기\n도하였다.\n②산과고개\n소백산맥의남단에위치한지리산은전라북도남원시, 전라남도구례군, 경상남도함양군·'),
  0.28755797080611517),
 (Document(page_content='역에속한다. 소백산맥(小白山脈)의지맥으로서추풍령(秋風嶺) 부근에서남서방향으로갈라\n진다. 주요산은운장산(1,126m), 마이산(678m), 내장산신선봉(763m), 방장산(734m), 태청산\n(593m), 불갑산(516m) 등으로저산성산지를이룬다. 이산맥과소백산맥의사이에는남서방\n향으로폭10~20km의지구상(地溝狀) 고원이전개되어있다. 이곳을경계로금강이북류하'),
  0.29455445220907156),
 (Document(page_content='16 제1부전라·제주권의지명\n산청군·하동군등3개도, 5개시·군에걸쳐위치하고있으며, 최고봉은천왕봉(1,915m)이\n다. 우리나라5대명산중하나로, 웅장하고경치가뛰어나국립공원제1호로지정되어있다.\n신라시대오악(五嶽) 중남악(南嶽)에해당하는산이다. 주능선을중심으로각각남북으로큰\n그림1. 전라권의하천과주요산'),
  0.31036115602255737)]

In [91]:
max_similarity_doc = min(result, key=lambda x: x[1])[0].page_content
max_similarity_doc

'었다. \n민주지산에서지리산으로이어지는소백산맥의서사면산지가전라남도의동쪽산지를형\n성한다. 소백산맥은태백산에서태백산맥과갈라져민주지산에이르기전까지경상도와충청\n도의경계를이루다가, 이후대덕산, 덕유산, 백운산, 지리산을거쳐여수반도에이른다. 금강,\n남한강수계와낙동강수계를나누고있으며, 역사적으로전라도의호남문화권과경상도의\n영남문화권을나누는경계역할을하였다.'

In [103]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 프롬프트 생성
prompt = f"""Answer the question based only on the following context, which can include texts:
{max_similarity_doc}
If you can't find the answer in the context, answer "I can't answer the question.".
Please answer in Korean.
"""

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    RunnablePassthrough(lambda x: prompt)  # 함수를 직접 전달
    | model
    | StrOutputParser()
)

# 파이프라인 실행
output = chain.invoke(query)
print(output)


소백산은 대한민국에 위치한 산으로, 충청북도와 경상북도의 경계에 걸쳐 있습니다. 구체적으로는 충청북도 단양군과 경상북도 영주시, 봉화군에 걸쳐 있습니다. 소백산은 이들 도와 시, 군에 걸쳐 있는 중요한 자연 명소로, 많은 등산객과 관광객이 찾는 곳입니다.


In [104]:
query = "지리산은 소백산의 어느 쪽에 위치하니?"
result = queryDB(query)
max_similarity_doc = min(result, key=lambda x: x[1])[0].page_content


# 프롬프트 생성
prompt = f"""Answer the question based only on the following context, which can include texts:
{max_similarity_doc}
If you can't find the answer in the context, answer "I can't answer the question.".
Please answer in Korean.
"""

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    RunnablePassthrough(lambda x: prompt)  # 함수를 직접 전달
    | model
    | StrOutputParser()
)

# 파이프라인 실행
output = chain.invoke(query)
print(output)


지리산은 소백산의 남쪽에 위치해 있습니다. 지리산은 대한민국 남부에 위치한 산으로, 전라남도, 전라북도, 경상남도에 걸쳐 있으며, 소백산은 충청북도와 경상북도에 걸쳐 있습니다. 따라서 지리산은 소백산보다 남쪽에 위치해 있습니다.
